In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
plt.style.use("ggplot")


## Data understanding

In [3]:
df_ais = pd.read_csv('../Project materials(1)/ais_train.csv', sep='|')
df_ais.head()
df_ais['time'] = pd.to_datetime(df_ais['time'])

df_ais.shape
#n = 1522065
#columns = 11
df_ais.columns
#latitude and longitude are our targets. Relevant covariates may be time, cog, sog, rot, heading, navstat, etaRAW, vesselId and portId.

df_ais.dtypes
#time, cog, latide and longitude are floats. Rot, heading and navstat are ints. etaRaw, vesselId and portId are objects. 

df_ais.describe()
#cog: course over ground. From 0 to 360 degrees.
#sog: speed over ground. from 0 to 1023 knots. 
#rot: rate of turning (of heading, which is the compass direction of where the boats bow/nose is heading). Degrees per minute.
#heading: direction of where boats bow is pointing. Measured in degrees from 0 to 360.
#navstat: Navigational status. The number tells the status of the boat. From 0 to 15.
#latitude: north-south position. Degrees. From -90 (south) to +90 (north). 
#longitude: east-west position. Degrees. From -180 (west) to +180 (east)

,time,cog,sog,rot,heading,navstat,latitude,longitude
count,1522065,1.522065e+06,1.522065e+06,1.522065e+06,1.522065e+06,1.522065e+06,1.522065e+06,1.522065e+06
mean,2024-03-06 03:20:23.657231360,1.782494e+02,6.331703e+00,5.054561e-01,1.762737e+02,2.092604e+00,3.658497e+01,1.153646e+01
min,2024-01-01 00:00:25,0.000000e+00,0.000000e+00,-1.270000e+02,0.000000e+00,0.000000e+00,-4.753287e+01,-1.675409e+02
25%,2024-02-03 02:59:19,7.820000e+01,0.000000e+00,0.000000e+00,7.500000e+01,0.000000e+00,3.451683e+01,-5.212810e+00
50%,2024-03-07 12:34:57,1.838000e+02,5.000000e-01,0.000000e+00,1.800000e+02,0.000000e+00,4.223370e+01,4.234350e+00
75%,2024-04-07 07:37:36,2.690000e+02,1.410000e+01,0.000000e+00,2.650000e+02,5.000000e+00,5.135197e+01,1.883715e+01
max,2024-05-07 23:59:08,3.600000e+02,1.023000e+02,1.280000e+02,5.110000e+02,1.500000e+01,7.055720e+01,1.788054e+02
std,NaN,1.076785e+02,7.539364e+00,1.760916e+01,1.070860e+02,2.411965e+00,2.290488e+01,6.871572e+01


## Data preparation

### Documentation of exploring estimated arrival times (don't run this!)

In [8]:
#Double-checking types of our columns.
#etaRAW should be a date since this is estimated time of arrival.
df_ais['etaRaw']

print(df_ais['etaRaw'][1:3])

def parse_time(raw_time):
    try:
    
        # Define the format without the year
        date_format = "%m-%d %H:%M"
    
        # Parse the cleaned string into a datetime object
        parsed_datetime = datetime.strptime(raw_time, date_format)

        # Add the correct year (2024)

        return parsed_datetime.replace(year = 2024)
    
    except ValueError:
        return None


df_ais['etaParsed'] = df_ais['etaRaw'].apply(parse_time)

#Want to ensure that the eta is the latest date. Checking how many instances I have of december.

december_count = df_ais[df_ais['etaParsed'].dt.month == 12].shape[0]

total_count = df_ais.shape[0]


print(f"Number of instances in December: {december_count/total_count}")

#Checking if I have instances of the etaParsed is later than time.

df_ais['later'] = df_ais['etaParsed'] > df_ais['time']

#November and later:
df_ais['later'] = df_ais['etaParsed'] > df_ais['time']

df_coolio = df_ais[df_ais['later'] == True].copy()

#Checking if there is any instances where etaParsed is in november
df_coolio[df_coolio['vesselId'] == '61e9f3a8b937134a3c4bfdf7'].head(10)

df_coolio_nov = df_coolio[df_coolio['etaParsed'].dt.month == 9].copy()

df_coolio_nov[df_coolio_nov['vesselId'] == '61e9f42eb937134a3c4c0103'].tail(40)

df_ais[df_ais['vesselId'] == '61e9f42eb937134a3c4c0103'].head(10)

#NOt fruitful

"""

later_count = df_ais[df_ais['later']  == False].shape[0]

print(f"Number of instances where time is later than eta: {later_count}")
df_ais[df_ais['later']  == False].head()

#I have instances where time is later than eta if I do this solution. 

#So when later is False I want to ensure that the year of etaParsed is 2025.

def adjust_year(time1,time2):
    if time1 <= time2:
        return time1.replace(year = 2025)
    return time1

df_ais['etaParsed'] = df_ais.apply(lambda row: adjust_year(row['etaParsed'], row['time']), axis = 1)
df_ais[df_ais['later']  == False].head() #It's fixed
yess
"""



1    12-29 20:00
2    01-02 09:00
Name: etaRaw, dtype: object
Number of instances in December: 0.01564913456389839


'\n\nlater_count = df_ais[df_ais[\'later\']  == False].shape[0]\n\nprint(f"Number of instances where time is later than eta: {later_count}")\ndf_ais[df_ais[\'later\']  == False].head()\n\n#I have instances where time is later than eta if I do this solution. \n\n#So when later is False I want to ensure that the year of etaParsed is 2025.\n\ndef adjust_year(time1,time2):\n    if time1 <= time2:\n        return time1.replace(year = 2025)\n    return time1\n\ndf_ais[\'etaParsed\'] = df_ais.apply(lambda row: adjust_year(row[\'etaParsed\'], row[\'time\']), axis = 1)\ndf_ais[df_ais[\'later\']  == False].head() #It\'s fixed\nyess\n'

### Actual useful stuff

In [4]:
def parse_time(raw_time):
    try:
    
        # Define the format without the year
        date_format = "%m-%d %H:%M"
    
        # Parse the cleaned string into a datetime object
        parsed_datetime = datetime.strptime(raw_time, date_format)

        # Add placeholder year 2024.

        return parsed_datetime.replace(year = 2024)
    
    except ValueError:
        return None


df_ais['etaParsed'] = df_ais['etaRaw'].apply(parse_time)

#Sets all years to 2024, because not able to determine which years estimated arrival time should be.

In [5]:
df_ais.drop(['etaRaw'], axis = 1)

df_ais.isna().sum() #Estimated arrival time (1615 entries) and portId (1615 entries)

df_ais.loc[df_ais.duplicated()] #No duplicated rows



,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,etaParsed


In [7]:
df_ids = df_ais['vesselId']

df_ids = df_ids.drop_duplicates().copy()

df_ids.head()

df_ais.head(30)

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,etaParsed
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f,2024-01-09 23:00:00
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689,2024-12-29 20:00:00
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19,2024-01-02 09:00:00
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126,2024-12-31 20:00:00
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3,2024-01-25 12:00:00
5,2024-01-01 00:05:13,186.9,0.0,0,187,5,12-20 02:40,24.27431,-110.32727,61e9f468b937134a3c4c028f,61d37ac11366c3998241da0a,2024-12-20 02:40:00
6,2024-01-01 00:05:40,123.4,0.0,128,511,5,12-16 01:00,40.71466,29.46603,61e9f46bb937134a3c4c02b3,61d38259b7b7526e1adf3a41,2024-12-16 01:00:00
7,2024-01-01 00:05:49,151.2,0.0,0,20,5,12-31 18:30,-19.25026,146.83507,61e9f3bfb937134a3c4bfe9f,61d36f6e0a1807568ff9a115,2024-12-31 18:30:00
8,2024-01-01 00:06:18,265.0,0.1,0,122,1,12-30 19:00,-26.73068,153.29194,61e9f45bb937134a3c4c0221,61d36f640a1807568ff9a103,2024-12-30 19:00:00
9,2024-01-01 00:06:29,36.0,0.0,0,70,5,12-30 19:55,35.46922,139.68343,61e9f3e6b937134a3c4bff6d,61d379f61366c3998241d8d2,2024-12-30 19:55:00
